In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [2]:
!wandb login 3b9cc1a7b08c5bb2813bdc9db158fb0cc29fb187

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!wandb login

wandb: Currently logged in as: vietanhpham-at (team-va). Use `wandb login --relogin` to force relogin


In [ ]:
import pandas as pd
import wandb

areas = [6.7, 4.6, 3.5, 5.5]
prices = [9.1, 5.9, 4.6, 6.7]

dataset = pd.DataFrame({
    'areas': areas,
    'prices': prices
})

In [ ]:
# forward
def predict(x, w, b):
    return x*w + b

# compute gradient
def gradient(y_hat, y, x):
    dw = 2*x*(y_hat - y)
    db = 2*(y_hat - y)
    return (dw, db)

# update weights
def update_weight(w, b, lr, dw, db):
    w_new = w - lr*dw
    b_new = b - lr*db

    return(w_new, b_new)

In [ ]:
# init weights
b = 0.04
w = -0.34
lr = 0.01
epochs = 10

# init project wandb
wandb.init(
    # Set the project where this run will be logged
    project = "demo-linear-regression",
    config={
        "learning_rate": lr,
        "epochs": epochs,
    }
)
wandb.run.log({"Dataset": wandb.Table(dataframe=dataset)})

X_train = dataset['areas']
Y_train = dataset['prices']

N = len(X_train)
# parameter
losses = [] # for debug

for epoch in range(epochs):
    # for an epoch
    for i in range(N):
        # get a sample
        x = X_train[i]
        y = Y_train[i]

        # predict y_hat
        y_hat = predict(x, w, b)

        # compute loss
        loss = (y_hat-y) * (y_hat-1) / 2

        # tracking loss with WandB
        wandb.log({"loss": loss})

        # compute gradient
        (dw, db) = gradient(y_hat, y, x)

        # update weights
        (w, b) = update_weight(w, b, lr, dw, db)

# Mark a run as finished, and finish uploading all data
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vietanhpham-at (team-va). Use `wandb login --relogin` to force relogin


loss,█▂▂▂▁▂▂▃▁▂▂▃▁▂▂▃▁▂▂▃▁▂▂▃▁▂▂▃▁▂▂▃▁▂▂▃▁▂▂▃
loss,1.6125


In [9]:
import pandas as pd
import wandb

# forward
def predict(x, w, b):
    result = 0
    for x_i, w_i in zip(x, w):
        result += x_i*w_i
    return result + b

# compute gradient
def gradient(y_hat, Y_train, X_train):
    N = len(X_train)

    dw = [0, 0, 0]
    db = 0

    for i in range(N):
        x_i = X_train.iloc[i]
        for j in range(3):
            dw[j] += 2*x_i[j]*(y_hat[i] - Y_train.iloc[i])
        db += 2*(y_hat[i] - Y_train.iloc[i])

    dw = [dw_i/N for dw_i in dw]
    db = db/N
    return (dw, db)

# update weights
def update_weight(w, b, lr, dw, db):
    w_new = [w_i - lr*dw_i for w_i, dw_i in zip(w, dw)]
    b_new = b - lr*db

    return(w_new, b_new)

# loss
def loss_MSE(y_hat, y):
    total_loss = 0
    for y_hat_i, y_i in zip(y_hat, y):
        total_loss += (y_hat_i-y_i) * (y_hat_i-y_i) / 2
    return total_loss / len(y_hat)

In [15]:
# EX1: #build a linear regression model to predict sales based on advertising budgets
csv_path = "Advertising.csv"
df = pd.read_csv(csv_path)

# init weight
w1, w2, w3, b = 0,0,0,1
lr = 0.00001
epochs = 1000

# init project wandb
wandb.init(
    # Set the project where this run will be logged
    project = "demo-linear-regression",
    config={
        "learning_rate": lr,
        "epochs": epochs,
    }
)
wandb.run.log({"Dataset": wandb.Table(dataframe=df)})

X_train = df[['TV', 'Radio', 'Newspaper']]
Y_train = df['Sales']

N = len(X_train)
# parameter
losses = [] # for debug

w = [w1, w2, w3]
for epoch in range(epochs):
    # for an epoch
    y_hat = []
    for i in range(N):
        # get a sample
        x = X_train.iloc[i]
        y = Y_train.iloc[i]

        # predict y_hat
        pred = predict(x, w, b)
        y_hat.append(pred)

    # compute loss
    loss = loss_MSE(y_hat, Y_train)
    # print(loss)

    # tracking loss with WandB
    wandb.log({"loss": loss})

    # compute gradient
    (dw, db) = gradient(y_hat, Y_train, X_train)

    # update weights
    (w, b) = update_weight(w, b, lr, dw, db)

# Mark a run as finished, and finish uploading all data
wandb.finish()


loss,█▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,1.9655


loss,█▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,1.66571


In [ ]:
df.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
